In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from BidderModel.Bid import bid, robust_bid_mse
from Utils.NoiseCTR import noise_CTR_mse
from Utils.Vizualization import plot_results_with_std,visualize_comparison
from BidderModel.IPinYou import BidGenerator,process_ipinyou_files


In [ ]:
bid_generator = BidGenerator()

file_list = ['IPinYouData/bid_ipinyou_19.txt', 'IPinYouData/bid_ipinyou_11.txt', 'IPinYouData/bid_ipinyou.txt','IPinYouData/bid_ipinyou_25.txt', 'IPinYouData/bid_ipinyou_28.txt']
bid_data = process_ipinyou_files(file_list)

In [ ]:
def safe_mean(arr):
    return np.mean(arr) if len(arr) > 0 else 0

def safe_std(arr):
    return np.std(arr) if len(arr) > 0 else 0

def experiment_2(SEED,epsilon,robustness,bid_data):
    I = 10  
    T = 100
    alpha=math.sqrt(2*epsilon)
    np.random.seed(SEED)

    CTRmin=0.01
    CTRmax=0.1
    CTR_real = np.random.uniform(CTRmin, CTRmax, (I, T))
    CVR = np.random.uniform(0.1, 0.9, (I, T))
    CTR_noised = noise_CTR_mse(I,T,CTR_real,epsilon,SEED,CTRmin,CTRmax)
    B = np.random.uniform(2500,2501, size=I)
    C = np.full(I, 5000.0)
    is_win = np.zeros((I, T))
    BID = np.zeros((I, T))
    p = np.ones((I, T))
    q = np.ones((I, T))
    wp = np.zeros(T)
    current_budgets= np.tile(B.reshape(-1, 1), T)
    advertiser_id = list(bid_data.keys())
    N_resolve=1
    N_solve=1000
   
    for i in range(0,9):
        BID[i]=bid_generator.generate_bid(advertiser_id=advertiser_id[i], N_bids=T,  bid_data=bid_data)
    
    bids = np.zeros(I)
    
    for t in range(T):
        
        for i in range(0,9):
            bids[i]=min(BID[i,t],current_budgets[i,t])
        if t == 0:
          
            bids[9] =300
        else:
            
            i=9
            pqmax=0.0001
            if robustness=='non-robust':
            
                bids[i],p[i,t],q[i,t] = bid(CTR_noised[i,:t], CVR[i,:t], wp[:t], B[i], alpha, C[i], 
                                  is_win[i,:t], np.sum(is_win[i,:t]),pqmax,p[i,t-1],q[i,t-1],N_resolve,N_solve)
                bids[i]=min(bids[i],current_budgets[i,t])
                
            else:   
                bids[i],p[i,t],q[i,t] =robust_bid_mse(CTR_noised[i,:t], CVR[i,:t], wp[:t], B[i], alpha, C[i], 
                                      is_win[i,:t], np.sum(is_win[i,:t]),pqmax,p[i,t-1],q[i,t-1],N_resolve,N_solve)
                bids[i]=min(bids[i],current_budgets[i,t])
            if bids[i]==-1:
                bids[i]=min(BID[i,t-1],current_budgets[i,t])
                p[i,t],q[i,t]=p[i,t-1],q[i,t-1]
                
        BID[:,t] = bids
        winner = np.argmax(bids)
     
        wp[t] = bids[winner]
        is_win[winner,t] = 1
        current_budgets[winner,t+1:] -= wp[t]
 
    total_conversions = np.sum(CTR_real[-1] * CVR[-1] * is_win[-1])
    
    cheap_clicks_ratio = []
    wins = is_win[-1] > 0
    if np.sum(wins) > 0:
        cheap_clicks = np.sum((BID[-1]/CTR_real[-1] < C[-1]) * CTR_real[-1] * is_win[-1])
        total_clicks = np.sum(CTR_real[-1] * is_win[-1])
        cheap_clicks_ratio.append(cheap_clicks/total_clicks if total_clicks > 0 else 0)
    avg_cheap_clicks = safe_mean(cheap_clicks_ratio)
    total_cost = np.sum(BID[-1] * is_win[-1])
    total_clicks = np.sum(CTR_real[-1] * is_win[-1])
    avg_cpc = total_cost / total_clicks if total_clicks > 0 else 0

   
    return (total_conversions,avg_cheap_clicks*100,avg_cpc)

In [ ]:
def run_experiments(robust_type,bid_data):
    results = []
    all_results=[]
    for epsilon in  tqdm([0.000001,0.0001,0.0005,0.001,0.005,0.01, 0.05,0.1]):

        print(epsilon)
        conversions = []
        clicks = []
        cpcs = []
        
        for SEED in range(0,100):
            total_conversions, avg_cheap_clicks, avg_cpc = experiment_2(SEED, epsilon, robust_type,bid_data)
            conversions.append(total_conversions)
            clicks.append(avg_cheap_clicks)
            cpcs.append(avg_cpc)
            all_results.append({
                'epsilon': epsilon,
                'conversions': total_conversions,
                'cheap_clicks': avg_cheap_clicks,
                'cpc': avg_cpc,
                'SEED':SEED
            }) 
            
        results.append({
            'epsilon': epsilon,
            'total_conversions_mean': safe_mean(conversions),
            'total_conversions_std': safe_std(conversions),
            'avg_cheap_clicks_mean': safe_mean(clicks),
            'avg_cheap_clicks_std': safe_std(clicks),
            'avg_cpc_mean': safe_mean(cpcs),
            'avg_cpc_std': safe_std(cpcs)
        })
   
    df = pd.DataFrame(results)
    df.to_csv(f'Results_ipinyou_{robust_type}_mse.csv', index=False)
    df1 = pd.DataFrame(all_results)
    df1.to_csv(f'All_results_ipinyou_{robust_type}_mse.csv', index=False)

run_experiments('non-robust',bid_data)
run_experiments('robust',bid_data)

In [ ]:
plot_results_with_std('Results_ipinyou_non-robust_mse.csv','Results_ipinyou_robust_mse.csv','Results_ipinyou_mse.png')